In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [2]:
# Paramètre à modifier : budget total (en M£, ou multiplie x10 si prix en 0.1M£)
my_budget = 1007

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Pondérations pour chaque critère (à ajuster selon l’importance donnée à chaque indicateur)
w_points_per_cost = 0.2  # positif : favorise joueurs rapportant beaucoup de points par rapport à leur coût
w_ict_index = 0.1
w_minutes_last = 0.1 # positif : favorise joueurs ayant joué récemment
w_total_points_last = 0.2 # positif : favorise joueurs en forme
w_ict_last = 0.2   # positif : favorise joueurs en forme
w_xGI_last = 0.1  # Poids ajouté sur xG_recent pour MID et FWD
w_goals_conceded_per_90 = -0.2 # négatif : pénalise DEF et GKP concédant beaucoup de buts
w_saves_per_90 = 0.1 # Poids ajouté sur saves pour DEF et GKP
w_clean_sheets_per_90 = 0.1 # Poids ajouté sur clean_sheets_per_90 pour DEF et GKP
w_defensive_contribution_per_90 = 0.1 # Poids ajouté sur defensive_contribution_per_90 pour DEF et GKP
w_fdr = -0.4   # négatif : pénalise calendrier difficile
w_total_points_last_per_xGI_last = -0.0 # négatif : met en avant les joueurs sous-côtés

# Variable d’état pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : score multi-indicateurs pondéré
prob = LpProblem("FPL_Optimizer", LpMaximize)
prob += lpSum([
    w_points_per_cost  * df_players.loc[i, 'points_per_cost'] * player_vars[i]
  + w_ict_index       * df_players.loc[i, 'ict_index'] * player_vars[i]
  + w_minutes_last * df_players.loc[i, 'minutes_last'] * player_vars[i]
  + w_total_points_last * df_players.loc[i, 'total_points_last'] * player_vars[i]
  + w_ict_last       * df_players.loc[i, 'ict_last'] * player_vars[i]
  + (w_goals_conceded_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['goals_conceded_per_90'] * player_vars[i]
  + (w_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.iloc[i]['xGI_last'] * player_vars[i]
  + (w_saves_per_90 if df_players.iloc[i]['position'] in ['GKP'] else 0) * df_players.iloc[i]['saves_per_90'] * player_vars[i]
  + (w_clean_sheets_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['clean_sheets_per_90'] * player_vars[i]
  + (w_defensive_contribution_per_90 if df_players.iloc[i]['position'] in ['GKP', 'DEF'] else 0) * df_players.iloc[i]['defensive_contribution_per_90'] * player_vars[i]
  + w_fdr             * df_players.loc[i, 'fdr_next_6'] * player_vars[i]
  + (w_total_points_last_per_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.loc[i, 'total_points_last_per_xGI_last'] * player_vars[i]
    for i in range(len(df_players))
    if player_vars[i] == 1
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte de budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= my_budget

# Contrainte max 3 joueurs de la même équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i] for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Résolution
prob.solve()

# Résultats : joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/d2641de2381d423ab9ae67c2794db83f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/d2641de2381d423ab9ae67c2794db83f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 5226 RHS
At line 5253 BOUNDS
At line 5996 ENDATA
Problem MODEL has 26 rows, 742 columns and 2968 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 218.693 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 26 rows, 623 columns (623 integer (552 of which binary)) and 2492 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -218.693
Cbc0038I Cleaned solution of -218.693
Cbc0038I Before

In [3]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,clean_sheets_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
0,Omar,Alderete,SUN,Sunderland,DEF,40.0,39.0,0.975000,0.080082,3.000000,29.8,3.4,100,8.5,487.0,332762,102683,69772,2378,1.0,1.0,2.0,0.58,0.33,0.91,2.197802,0.11,0.00,0.06,0.17,1.21,0.74,10.90,0.37,90.000000,0.216667,0.156667,6.166667,10.000000,46.153846
1,Marcos,Senesi,BOU,Bournemouth,DEF,48.0,44.0,0.916667,0.081481,3.166667,28.1,19.6,19,8.2,540.0,2301562,179351,176830,4074,2.0,0.0,2.0,0.14,0.37,0.51,3.921569,0.02,0.00,0.06,0.08,0.91,1.17,13.50,0.50,90.000000,0.093333,0.000000,5.200000,6.333333,67.857143
7,Tyrick,Mitchell,CRY,Crystal Palace,DEF,50.0,34.0,0.680000,0.062963,3.166667,32.7,2.1,126,6.0,540.0,156524,18816,77513,2431,0.0,1.0,1.0,0.28,0.46,0.74,1.351351,0.05,0.00,0.08,0.13,1.14,0.50,8.50,0.50,90.000000,0.210000,0.093333,6.866667,6.000000,28.571429
13,Jurriën,J.Timber,ARS,Arsenal,DEF,58.0,37.0,0.637931,0.085847,2.500000,43.2,14.4,26,3.2,431.0,1510997,55716,339705,17229,1.0,2.0,3.0,1.56,0.38,1.94,1.546392,0.33,0.00,0.08,0.41,0.56,0.63,7.52,0.42,86.333333,0.313333,0.220000,6.500000,3.666667,11.702128
22,Gabriel,Gabriel,ARS,Arsenal,DEF,62.0,38.0,0.612903,0.070370,2.500000,24.4,24.2,13,6.5,540.0,1242879,88921,621082,15888,0.0,1.0,1.0,0.92,0.06,0.98,1.020408,0.15,0.00,0.01,0.16,0.63,0.50,8.67,0.50,90.000000,0.300000,0.286667,5.833333,8.000000,26.666667
48,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,37.0,0.474359,0.072549,2.666667,40.2,66.8,1,5.0,510.0,2684726,30266,1185016,162572,3.0,2.0,5.0,1.50,0.23,1.73,2.890173,0.26,0.00,0.04,0.30,1.27,1.24,4.94,0.35,86.000000,0.076667,0.050000,4.000000,3.666667,47.826087
66,Erling,Haaland,MCI,Man City,FWD,144.0,62.0,0.430556,0.123260,3.333333,70.6,52.2,3,11.8,503.0,3554684,336425,614231,1845,1.0,8.0,9.0,7.35,0.56,7.91,1.137800,1.32,0.00,0.10,1.42,1.14,0.89,4.29,0.54,83.666667,1.250000,1.163333,13.933333,12.666667,10.133333
169,Jean-Philippe,Mateta,CRY,Crystal Palace,FWD,75.0,18.0,0.240000,0.033898,3.166667,32.5,9.9,42,3.8,531.0,1020736,38625,920912,29995,0.0,2.0,2.0,3.59,0.44,4.03,0.496278,0.61,0.00,0.07,0.68,1.11,0.51,5.76,0.51,90.000000,0.820000,0.783333,6.866667,3.333333,4.065041
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,39.0,0.866667,0.072222,3.000000,19.8,3.1,105,7.5,540.0,355480,48166,90379,2415,0.0,0.0,0.0,0.00,0.01,0.01,0.000000,0.00,3.33,0.00,0.00,1.15,0.67,0.00,0.50,90.000000,0.003333,0.000000,3.900000,7.333333,2200.000000
8,Nick,Pope,NEW,Newcastle,GKP,50.0,34.0,0.680000,0.062963,3.000000,13.5,7.2,55,5.2,540.0,528160,44684,199735,6005,1.0,0.0,1.0,0.00,0.00,0.00,0.000000,0.00,2.50,0.00,0.00,0.78,0.83,0.00,0.67,90.000000,0.000000,0.000000,2.933333,5.333333,0.000000


In [4]:
selected_df['price'].sum()

np.float64(1006.0)

In [5]:
selected_df['selected_by_percent'].sum()

np.float64(317.29999999999995)